In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


"""

Coding Task:

    Fine-tune BERT on a small dataset (e.g., IMDb movie reviews for sentiment classification).
    Use transformers and Trainer from Hugging Face.

Hint: Use the datasets library to load IMDb:
"""
import torch
import transformers
from datasets import load_dataset, DatasetDict
from transformers import DistilBertTokenizer, DataCollatorWithPadding, DistilBertForSequenceClassification, TrainingArguments, \
    Trainer
import wandb

MODEL_NAME = "distilbert-base-uncased" # "bert-base-uncased"

if __name__ == '__main__':
    isGpuAvailable = torch.cuda.is_available()
    print(f"{isGpuAvailable = }")  # Should print: True
    print(f"{torch.cuda.device_count() = }")  # Should print: 1 (if you have one GPU)
    if isGpuAvailable:
        print(f"{torch.cuda.get_device_name(0) = }")  # on my laptop, it should print: NVIDIA GeForce RTX 2060 Max-Q. on kaggle, it prints tesla t4
    else:
        print("No gpu found!")

    transformers.logging.set_verbosity_debug()  # Set to 'INFO' for fewer logs
    wandb.init(mode="offline")  # Logs only locally

    imdb_dataset = load_dataset("imdb")

    imdb_dataset = DatasetDict({        # keep only what I need
        "train": imdb_dataset["train"],
        "test": imdb_dataset["test"]
    })



    print(f"{imdb_dataset = }")
    # 2 columns, text, and label

    # load tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

    # we need a tokenizer function. for now follow tutorial. java styled code organization later
    def tokenize_function(example):
        try:
            return tokenizer(example["text"], padding="max_length", truncation=True)
        except Exception as e:
            print(f"Error in tokenization: {e}")
            return None  # Or handle differently

    tokenized_dataset = imdb_dataset.map(tokenize_function, batched=True)

    print(f"{tokenized_dataset = }")
    """
    tokenized_dataset = DatasetDict({
        train: Dataset({
            features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
            num_rows: 25000
        })
        test: Dataset({
            features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
            num_rows: 25000
        })
        unsupervised: Dataset({
            features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
            num_rows: 50000
        })
    })    
    """

    # convert data to pytorch format
    print("creating tokenized_dataset")
    tokenized_dataset = tokenized_dataset.remove_columns(["text"]) # we don't need text column
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels") # huggingface library wants y = labels
    tokenized_dataset.set_format("torch") # convert matrices into pytorch

    print("creating DataCollatorWithPadding")
    # Data collator for padding batches dynamically
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # Question: What is data collector? / what does it do?

    # load the model
    print("creating Model")
    model = (DistilBertForSequenceClassification
             .from_pretrained(MODEL_NAME, num_labels = 2))

    if isGpuAvailable:
        print("before setting model to cuda")
        model = model.to("cuda")  # <-- Ensure Trainer runs on GPU
        print("after setting model to cuda")

    # define training arguments, and trainer
    BATCH_SIZE = 32
    print("creating train_args")
    train_args = TrainingArguments(
        run_name="bert_imdb_experiment",  # Set a custom run name to repair wandDb warning
        output_dir="./bert-imdb",
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        # device="cuda" # looks like different version had this param
    )

    # question: what about optimizer, loss function?

    # initialize the trainer
    print("creating trainer")
    trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        # tokenizer=tokenizer, # tokenizer is deprecated. use data_collector, or process_class instead :/
        data_collator=data_collator,
        # device="cuda" # looks like different version had this param
    )

    # the training!
    print("trainer.train()!")
    trainer.train() # this will fine tune the dataset for 3 epochs!
    print("trainer.evaluate()!")
    trainer.evaluate() # evaluate

    def predict_sentiment(text):
        tokenized_text = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():    # forgot the braces. 
            outputs = model(**tokenized_text)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()
        return "Positive" if prediction == 1 else "Negative"

    print("predict_statement")
    print(predict_sentiment("I really loved this movie! It was fantastic."))
    print(predict_sentiment("This was the worst movie I have ever seen."))

    pass


isGpuAvailable = True
torch.cuda.device_count() = 2
torch.cuda.get_device_name(0) = 'Tesla T4'


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...


***** Running Evaluation *****
  Num examples = 25000
  Batch size = 64
Saving model checkpoint to ./bert-imdb/checkpoint-391
Configuration saved in ./bert-imdb/checkpoint-391/config.json
Model weights saved in ./bert-imdb/checkpoint-391/model.safetensors
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(

***** Running Evaluation *****
  Num examples = 25000
  Batch size = 64
Saving model checkpoint to ./bert-imdb/checkpoint-782
Configuration saved in ./bert-imdb/checkpoint-782/config.json
Model weights saved in ./bert-imdb/checkpoint-782/model.safetensors
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Saving model checkpoint to ./bert-im

trainer.evaluate()!


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


predict_statement


AttributeError: __enter__